# Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy
import scipy.stats as st
from sklearn.decomposition import PCA
from lmfit import minimize, Parameters
from tqdm.notebook import tqdm

mpl.rcParams["font.sans-serif"] = "Arial"
mpl.rcParams["font.family"] = "sans-serif"
plt.rcParams["figure.figsize"] = (10, 6)
mpl.rcParams["pdf.fonttype"] = 42
sns.set_style(
    "ticks",
    {
        "xtick.major.size": 4,
        "ytick.major.size": 4,
        "font_color": "k",
        "axes.edgecolor": "k",
        "xtick.color": "k",
        "ytick.color": "k",
    },
)
sns.set_context("talk", font_scale=1.0)

act_rep_palette = ["tab:blue", "tab:orange", "tab:olive"]

d2_threshold = 0.639
d5_threshold = 0.103

d2_baseline_threshold = -0.938
d5_baseline_threshold = -0.300

# Importing Data

In [2]:
df = pd.read_csv("../../fig_2/01_activators_synergy/pairs_baselinesums.csv")
oligos = pd.read_csv("../../fig_1/01_raw_counts/csvs/base_oligo_library.csv")

# Fitting Data

In [3]:
def domain_line_fit(gene, ax, debug=False):
    def printdb(s):
        if debug:
            print(s)

    # get the data
    domain = list(oligos[oligos["Gene"] == gene]["label"])[0]
    printdb("Test domain:\t" + domain)
    smoldf = df[(df["domain1"] == domain) | (df["domain2"] == domain)]
    if smoldf.shape[0] == 0:
        print()
        print()
        print("No entries for", gene)
        print("Returning empty dataframe, not plotting")
        print()
        return pd.DataFrame.from_dict({"gene": [], "label": [], "m": [], "b": []})
    printdb("Length of smol df:\t" + str(smoldf.shape))
    that_scores = np.where(
        smoldf["domain1"] == domain, smoldf["d2_med_d5"], smoldf["d1_med_d5"]
    )
    this_scores = np.where(
        smoldf["domain1"] == domain, smoldf["d2_med_d5"], smoldf["d1_med_d5"]
    )
    combo_scores = np.array(smoldf["avg_enrichment_d5"])
    test_df = pd.DataFrame.from_dict(
        {"domain": this_scores, "other": that_scores, "combo": combo_scores}
    )

    # fit the line
    filt_df = test_df.dropna()

    def line(x, m, b):
        return m * x + b

    def residual(params, x, data):
        m = params["m"]
        b = params["b"]
        model = line(x, m, b)
        return data - model

    params = Parameters()
    params.add("m", min=-20, max=20)
    params.add("b", min=-20, max=20)
    out = minimize(
        residual, params, args=(filt_df["other"], filt_df["combo"]), method="nelder"
    )

    # plot
    ax.set_xlim(-3, 3)
    ax.set_ylim(-5.5, 5)
    m = out.params["m"].value
    b = out.params["b"].value
    g = sns.scatterplot(data=filt_df, x="other", y="combo", ax=ax, marker=".")
    ax.set_title(gene)
    ax.set_xlabel("Partner Repression $\log_2$(ON:OFF)")
    ax.set_ylabel("Combo Repression\n$\log_2$(ON:OFF)")
    x = np.linspace(-5.3, 5.3, 500)
    y = line(x, m, b)
    ax.plot(x, y, color="tab:red")

    return pd.DataFrame.from_dict(
        {"gene": [gene], "label": [domain], "m": [m], "b": [b]}
    )

In [4]:
rd1df = df[
    (df["d1_baseline_type"].isin(["Repressor", "Dual"])) | (df["d1_description"].str.contains("Rep"))
]
repressors = sorted(list(set(list(rd1df["d1_Gene"]))))

df_list = []

for g in tqdm(repressors):
    fig, ax = plt.subplots(figsize=(4, 2))
    df_list.append(domain_line_fit(g, ax))
    sns.despine()
    fig.savefig("./fitplots/" + g + ".pdf", bbox_inches="tight")
    plt.close("all")

linedf = pd.concat(df_list)
linedf

  0%|          | 0/52 [00:00<?, ?it/s]

,gene,label,m,b
0,ADRM1,Silencer_tiles;ENSG00000130706;26,0.843589,0.165043
0,ASCL1,Short_nuclear_domain;ASCL1_HUMAN;HLH;105;52,0.222941,-1.006156
0,ATF1,Short_nuclear_domain;ATF1_HUMAN;bZIP_1;191;59,0.629781,-1.074953
0,ATRX,Silencer_tiles;ENSG00000085224;242,1.055076,-0.109748
0,BAZ2A,Silencer_tiles;ENSG00000076108;118,0.505857,-1.472036
0,BIN1,Short_nuclear_domain;BIN1_HUMAN;SH3_9;513;63,0.873128,-0.397710
0,CACO1,Short_nuclear_domain;CACO1_HUMAN;Zn-C2H2_12;61...,0.644867,-0.345332
0,CBX1,Short_nuclear_domain;CBX1_HUMAN;Chromo_shadow;...,0.111754,-2.325725
0,CBX7,Silencer_tiles;ENSG00000100307;18,0.325750,-1.774144
0,CDYL2,Short_nuclear_domain;CDYL2_HUMAN;Chromo;1;50,0.202063,-2.678337


# Merge with prior data

In [5]:
bdf = pd.read_csv("../../fig_1/03_computing_baselines/baseline_scores.csv")
bdf["label"] = bdf["domain"]
bdf.head()

,domain,avg_d2,med_d2,sd_d2,min_d2,max_d2,delta_min_d2,delta_max_d2,avg_d5,med_d5,...,rep_min_d5,rep_max_d5,rep_delta_min_d5,rep_delta_max_d5,is_activator,is_repressor,baseline_type,prior_type,gene,label
0,Short_nuclear_domain;CBX1_HUMAN;Chromo_shadow;...,-2.723895,-2.796018,0.688257,-3.899350,-1.152775,1.103332,2.746575,-2.581530,-2.642873,...,-3.796554,-1.440654,1.046026,2.355901,False,True,Repressor,Repressor,NaN,Short_nuclear_domain;CBX1_HUMAN;Chromo_shadow;...
1,Short_nuclear_domain;SUMO3_HUMAN;Rad60-SLD;12;70,-2.559898,-2.672776,0.499699,-3.213500,-1.414590,0.540724,1.798910,-2.594383,-2.608911,...,-4.673955,-0.704240,2.049007,3.969714,False,True,Repressor,Repressor,NaN,Short_nuclear_domain;SUMO3_HUMAN;Rad60-SLD;12;70
2,Short_nuclear_domain;CDYL2_HUMAN;Chromo;1;50,-2.537211,-2.326746,0.790697,-3.818794,-0.558255,1.492047,3.260539,-2.876315,-3.294145,...,-4.160111,-0.947768,0.809262,3.212343,False,True,Repressor,Repressor,NaN,Short_nuclear_domain;CDYL2_HUMAN;Chromo;1;50
3,Short_nuclear_domain;MPP8_HUMAN;Chromo;44;50,-2.639417,-2.695604,0.448160,-3.269960,-1.850474,0.574355,1.419486,-2.251854,-2.699218,...,-3.946956,-0.696035,1.124470,3.250921,False,True,Repressor,Repressor,NaN,Short_nuclear_domain;MPP8_HUMAN;Chromo;44;50
4,Short_nuclear_domain;YAF2_HUMAN;YAF2_RYBP;78;32,-1.868021,-2.050970,0.917464,-2.977203,0.840593,0.926233,3.817796,-2.509226,-2.996155,...,-4.032861,-0.533026,0.984842,3.499835,False,True,Repressor,Repressor,NaN,Short_nuclear_domain;YAF2_HUMAN;YAF2_RYBP;78;32


In [6]:
adf = bdf.merge(linedf, on="label", how="left").reset_index()
adf['gene'] = adf['gene_x']
adf = adf[["domain", "gene", "med_d5", "m", "b"]]
adf = adf.dropna(subset=["m", "b"])
adf.to_csv("./repressors_fitted.csv", index=False)
adf.head()

,domain,gene,med_d5,m,b
0,Short_nuclear_domain;CBX1_HUMAN;Chromo_shadow;...,NaN,-2.642873,0.111754,-2.325725
1,Short_nuclear_domain;SUMO3_HUMAN;Rad60-SLD;12;70,NaN,-2.608911,0.079367,-2.471963
2,Short_nuclear_domain;CDYL2_HUMAN;Chromo;1;50,NaN,-3.294145,0.202063,-2.678337
3,Short_nuclear_domain;MPP8_HUMAN;Chromo;44;50,NaN,-2.699218,0.291206,-2.260485
4,Short_nuclear_domain;YAF2_HUMAN;YAF2_RYBP;78;32,NaN,-2.996155,0.143072,-2.259422
